In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stresult22-15-22.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 13
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [3]:
#we store the variables here
log_files_path = 'C:/Users/jira_/Desktop/new_st17-13-17/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [15, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 15], [15], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '10'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 15]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/new_st17-13-17/tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,15)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 1.564336598
Validation Loss: 2.1406465
Epoch: 0002 cost = 1.549837152
Validation Loss: 1.8441361
Epoch: 0003 cost = 1.535562237
Validation Loss: 1.6761758
Epoch: 0004 cost = 1.521159033
Validation Loss: 1.6104422
Epoch: 0005 cost = 1.506055931
Validation Loss: 1.5893092
Epoch: 0006 cost = 1.489448786
Validation Loss: 1.5925173
Epoch: 0007 cost = 1.471017698
Validation Loss: 1.6615874
Epoch: 0008 cost = 1.452412069
Validation Loss: 1.6937116
Epoch: 0009 cost = 1.435415367
Validation Loss: 1.7183766
Epoch: 0010 cost = 1.419931034
Validation Loss: 1.7242548
Epoch: 0011 cost = 1.406932533
Validation Loss: 1.683857
Epoch: 0012 cost = 1.394976477
Validation Loss: 1.653786
Epoch: 0013 cost = 1.382655521
Validation Loss: 1.6518224
Epoch: 0014 cost = 1.370206277
Validation Loss: 1.6540656
Epoch: 0015 cost = 1.358120282
Validation Loss: 1.6556565
Epoch: 0016 cost = 1.346222162
Validation Loss: 1.6919254
Epoch: 0017 cost = 1.334326665
Validation Loss: 1.7647476
Epoch: 

Validation Loss: 0.6654295
Epoch: 0159 cost = 0.536627034
Validation Loss: 0.6397527
Epoch: 0160 cost = 0.533347517
Validation Loss: 0.65960544
Epoch: 0161 cost = 0.530087928
Validation Loss: 0.6521028
Epoch: 0162 cost = 0.526848137
Validation Loss: 0.6532384
Epoch: 0163 cost = 0.523627718
Validation Loss: 0.6369968
Epoch: 0164 cost = 0.520427162
Validation Loss: 0.6248786
Epoch: 0165 cost = 0.517245317
Validation Loss: 0.63613284
Epoch: 0166 cost = 0.514083132
Validation Loss: 0.6217539
Epoch: 0167 cost = 0.510939429
Validation Loss: 0.62298995
Epoch: 0168 cost = 0.507814502
Validation Loss: 0.6018185
Epoch: 0169 cost = 0.504708429
Validation Loss: 0.6291692
Epoch: 0170 cost = 0.501621127
Validation Loss: 0.62488717
Epoch: 0171 cost = 0.498551711
Validation Loss: 0.6226214
Epoch: 0172 cost = 0.495500927
Validation Loss: 0.5952349
Epoch: 0173 cost = 0.492468263
Validation Loss: 0.5683905
Epoch: 0174 cost = 0.489454170
Validation Loss: 0.5852815
Epoch: 0175 cost = 0.486457681
Validation

Epoch: 0309 cost = 0.209549956
Validation Loss: 0.2647248
Epoch: 0310 cost = 0.208198498
Validation Loss: 0.28095588
Epoch: 0311 cost = 0.206856544
Validation Loss: 0.27697653
Epoch: 0312 cost = 0.205523533
Validation Loss: 0.26951432
Epoch: 0313 cost = 0.204198884
Validation Loss: 0.27686816
Epoch: 0314 cost = 0.202882916
Validation Loss: 0.2930756
Epoch: 0315 cost = 0.201574902
Validation Loss: 0.27368698
Epoch: 0316 cost = 0.200274924
Validation Loss: 0.27497926
Epoch: 0317 cost = 0.198982748
Validation Loss: 0.28304508
Epoch: 0318 cost = 0.197698345
Validation Loss: 0.27522603
Epoch: 0319 cost = 0.196421643
Validation Loss: 0.2685537
Epoch: 0320 cost = 0.195152551
Validation Loss: 0.26938394
Epoch: 0321 cost = 0.193890482
Validation Loss: 0.27160093
Epoch: 0322 cost = 0.192636269
Validation Loss: 0.2699936
Epoch: 0323 cost = 0.191389191
Validation Loss: 0.27655172
Epoch: 0324 cost = 0.190149464
Validation Loss: 0.2786264
Epoch: 0325 cost = 0.188916740
Validation Loss: 0.27339235
Ep

Epoch: 0461 cost = 0.075710464
Validation Loss: 0.15301642
Epoch: 0462 cost = 0.075240986
Validation Loss: 0.15599623
Epoch: 0463 cost = 0.074775445
Validation Loss: 0.1359766
Epoch: 0464 cost = 0.074313539
Validation Loss: 0.1363165
Epoch: 0465 cost = 0.073855411
Validation Loss: 0.13006882
Epoch: 0466 cost = 0.073400831
Validation Loss: 0.13857566
Epoch: 0467 cost = 0.072949783
Validation Loss: 0.12713693
Epoch: 0468 cost = 0.072502642
Validation Loss: 0.13288118
Epoch: 0469 cost = 0.072058721
Validation Loss: 0.1378484
Epoch: 0470 cost = 0.071618245
Validation Loss: 0.14942229
Epoch: 0471 cost = 0.071181531
Validation Loss: 0.15086539
Epoch: 0472 cost = 0.070747984
Validation Loss: 0.17163225
Epoch: 0473 cost = 0.070317884
Validation Loss: 0.18786469
Epoch: 0474 cost = 0.069891054
Validation Loss: 0.19180311
Epoch: 0475 cost = 0.069467450
Validation Loss: 0.19974785
Epoch: 0476 cost = 0.069047093
Validation Loss: 0.23280993
Epoch: 0477 cost = 0.068629495
Validation Loss: 0.21431491


Epoch: 0609 cost = 0.036475683
Validation Loss: 0.0654223
Epoch: 0610 cost = 0.036367479
Validation Loss: 0.047725346
Epoch: 0611 cost = 0.036260421
Validation Loss: 0.047042653
Epoch: 0612 cost = 0.036154406
Validation Loss: 0.04445155
Epoch: 0613 cost = 0.036049776
Validation Loss: 0.058061343
Epoch: 0614 cost = 0.035946066
Validation Loss: 0.050790332
Epoch: 0615 cost = 0.035843565
Validation Loss: 0.05900043
Epoch: 0616 cost = 0.035742060
Validation Loss: 0.04462754
Epoch: 0617 cost = 0.035641740
Validation Loss: 0.044768
Epoch: 0618 cost = 0.035542407
Validation Loss: 0.04831677
Epoch: 0619 cost = 0.035444316
Validation Loss: 0.053857733
Epoch: 0620 cost = 0.035347152
Validation Loss: 0.06653507
Epoch: 0621 cost = 0.035251117
Validation Loss: 0.07069985
Epoch: 0622 cost = 0.035155870
Validation Loss: 0.068136565
Epoch: 0623 cost = 0.035061769
Validation Loss: 0.054244973
Epoch: 0624 cost = 0.034968640
Validation Loss: 0.05528776
Epoch: 0625 cost = 0.034876511
Validation Loss: 0.05

...

Epoch: 4334 cost = 0.013383613
Validation Loss: 0.03647207
Epoch: 4335 cost = 0.013451673
Validation Loss: 0.020024855
Epoch: 4336 cost = 0.013391442
Validation Loss: 0.023179382
Epoch: 4337 cost = 0.013468465
Validation Loss: 0.01905575
Epoch: 4338 cost = 0.013416129
Validation Loss: 0.032747075
Epoch: 4339 cost = 0.013508665
Validation Loss: 0.029938333
Epoch: 4340 cost = 0.013478439
Validation Loss: 0.022637676
Epoch: 4341 cost = 0.013588844
Validation Loss: 0.035018373
Epoch: 4342 cost = 0.013576231
Validation Loss: 0.036392204
Epoch: 4343 cost = 0.013666084
Validation Loss: 0.04766231
Epoch: 4344 cost = 0.013635696
Validation Loss: 0.04976867
Epoch: 4345 cost = 0.013799736
Validation Loss: 0.054592356
Epoch: 4346 cost = 0.013738407
Validation Loss: 0.038803637
Epoch: 4347 cost = 0.013654608
Validation Loss: 0.04672276
Epoch: 4348 cost = 0.013625834
Validation Loss: 0.054372363
Epoch: 4349 cost = 0.013724194
Validation Loss: 0.06860821
Epoch: 4350 cost = 0.013609968
Validation Loss

Validation Loss: 0.046632085
Epoch: 4484 cost = 0.013388789
Validation Loss: 0.041244388
Epoch: 4485 cost = 0.013445249
Validation Loss: 0.043573808
Epoch: 4486 cost = 0.013393682
Validation Loss: 0.023918686
Epoch: 4487 cost = 0.013454627
Validation Loss: 0.031567212
Epoch: 4488 cost = 0.013399076
Validation Loss: 0.02900175
Epoch: 4489 cost = 0.013464028
Validation Loss: 0.045837555
Epoch: 4490 cost = 0.013408785
Validation Loss: 0.029088056
Epoch: 4491 cost = 0.013486883
Validation Loss: 0.035302162
Epoch: 4492 cost = 0.013434847
Validation Loss: 0.033358954
Epoch: 4493 cost = 0.013513542
Validation Loss: 0.04904338
Epoch: 4494 cost = 0.013450011
Validation Loss: 0.020208089
Epoch: 4495 cost = 0.013519447
Validation Loss: 0.023571046
Epoch: 4496 cost = 0.013444135
Validation Loss: 0.03778026
Epoch: 4497 cost = 0.013503333
Validation Loss: 0.03659248
Epoch: 4498 cost = 0.013417350
Validation Loss: 0.026646413
Epoch: 4499 cost = 0.013473167
Validation Loss: 0.03383448
Epoch: 4500 cost

Validation Loss: 0.035948336
Epoch: 4632 cost = 0.013420112
Validation Loss: 0.044478334
Epoch: 4633 cost = 0.013527101
Validation Loss: 0.058998328
Epoch: 4634 cost = 0.013547561
Validation Loss: 0.069572315
Epoch: 4635 cost = 0.013643885
Validation Loss: 0.042255018
Epoch: 4636 cost = 0.013601770
Validation Loss: 0.03767588
Epoch: 4637 cost = 0.013687787
Validation Loss: 0.048451863
Epoch: 4638 cost = 0.013626116
Validation Loss: 0.039941054
Epoch: 4639 cost = 0.013550063
Validation Loss: 0.09024959
Epoch: 4640 cost = 0.013405536
Validation Loss: 0.08630169
Epoch: 4641 cost = 0.013399378
Validation Loss: 0.055634703
Epoch: 4642 cost = 0.013325125
Validation Loss: 0.032124605
Epoch: 4643 cost = 0.013364259
Validation Loss: 0.03646208
Epoch: 4644 cost = 0.013317201
Validation Loss: 0.043111358
Epoch: 4645 cost = 0.013369919
Validation Loss: 0.044287685
Epoch: 4646 cost = 0.013322503
Validation Loss: 0.041925266
Epoch: 4647 cost = 0.013375157
Validation Loss: 0.03604465
Epoch: 4648 cost

Epoch: 4788 cost = 0.013328997
Validation Loss: 0.032764133
Epoch: 4789 cost = 0.013370787
Validation Loss: 0.033365894
Epoch: 4790 cost = 0.013296621
Validation Loss: 0.029321944
Epoch: 4791 cost = 0.013352888
Validation Loss: 0.03505345
Epoch: 4792 cost = 0.013283230
Validation Loss: 0.042722672
Epoch: 4793 cost = 0.013344015
Validation Loss: 0.03585286
Epoch: 4794 cost = 0.013276970
Validation Loss: 0.030387986
Epoch: 4795 cost = 0.013366525
Validation Loss: 0.026845891
Epoch: 4796 cost = 0.013383645
Validation Loss: 0.039244663
Epoch: 4797 cost = 0.013488759
Validation Loss: 0.03412179
Epoch: 4798 cost = 0.013329729
Validation Loss: 0.02651173
Epoch: 4799 cost = 0.013429177
Validation Loss: 0.026724866
Epoch: 4800 cost = 0.013425524
Validation Loss: 0.039315414
Epoch: 4801 cost = 0.013555386
Validation Loss: 0.03424025
Epoch: 4802 cost = 0.013551979
Validation Loss: 0.034046896
Epoch: 4803 cost = 0.013648401
Validation Loss: 0.035452016
Epoch: 4804 cost = 0.013584901
Validation Los

Epoch: 4950 cost = 0.013398210
Validation Loss: 0.06653726
Epoch: 4951 cost = 0.013439810
Validation Loss: 0.03179402
Epoch: 4952 cost = 0.013395874
Validation Loss: 0.056472022
Epoch: 4953 cost = 0.013544134
Validation Loss: 0.044687588
Epoch: 4954 cost = 0.013552287
Validation Loss: 0.025978306
Epoch: 4955 cost = 0.013681355
Validation Loss: 0.032045845
Epoch: 4956 cost = 0.013592546
Validation Loss: 0.023585835
Epoch: 4957 cost = 0.013563989
Validation Loss: 0.054724228
Epoch: 4958 cost = 0.013435119
Validation Loss: 0.05535375
Epoch: 4959 cost = 0.013497504
Validation Loss: 0.05230353
Epoch: 4960 cost = 0.013484987
Validation Loss: 0.0423758
Epoch: 4961 cost = 0.013491233
Validation Loss: 0.036853906
Epoch: 4962 cost = 0.013424195
Validation Loss: 0.046673067
Epoch: 4963 cost = 0.013511272
Validation Loss: 0.056385055
Epoch: 4964 cost = 0.013474878
Validation Loss: 0.045051835
Epoch: 4965 cost = 0.013484502
Validation Loss: 0.038222276
Epoch: 4966 cost = 0.013361361
Validation Loss

In [4]:
import csv
with open('C:/Users/jira_/Desktop/new_stresult17-13-17.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))